In [ ]:
with open("../input.txt") as f:
    input = [line.strip() for line in f.readlines()]
# print(input)

with open("../sample-input.txt") as f:
    sample_input = [line.strip() for line in f.readlines()]
print(f"Input: {sample_input}")

In [ ]:
import numpy as np
grid_mask = {
    '.': 0,  # empty space
    '|': 1,  # vertical mirror
    '-': 2,  # horizontal mirror
    '/': 4,  # diagonal mirror leaning right
    '\\': 8  # diagonal mirror leaning left
}

def matrix_from_grid(grid):
    return np.array([[grid_mask[cell] for cell in row] for row in grid])

matrix_from_grid(sample_input)

In [31]:
g = matrix_from_grid(input)
display(g)

def counts_in_direction(g, direction):
       # Initialize an array to store the counts
       counts_vectorized = np.zeros_like(g)
       mask = (g == 0)
       counts_with_points = {}
       
       # For each identified location, calculate the counts
       if direction == 'right':
              # Create a mask for 0s and 2s
              mask |= (g == 2)
              result = np.argwhere((g > 1))
              result = [[0,0]] + result.tolist()
              for row, col in result:
                     if col < g.shape[1] - 1:
                            counts_vectorized[row, col] = min(g.shape[1] - col - 1, np.sum(mask[row, col+1:] & np.cumprod(mask[row, col+1:])) + 1)
                            counts_with_points[(row, col)] = (counts_vectorized[row, col], (row, col + counts_vectorized[row, col]))
       elif direction == 'left':
              mask |= (g == 2)
              result = np.argwhere((g > 1))
              for row, col in result:
                     if col > 0:
                            counts_vectorized[row, col] = min(col, np.sum(mask[row, :col] & np.cumprod(mask[row, :col][::-1])[::-1]) + 1)
                            counts_with_points[(row, col)] = (counts_vectorized[row, col], (row, col - counts_vectorized[row, col]))
       elif direction == 'up':
              mask |= (g == 1)
              result = np.argwhere((g > 0) & (g != 2))
              for row, col in result:
                     if row > 0:
                            counts_vectorized[row, col] = min(row, np.sum(mask[:row, col] & np.cumprod(mask[:row, col][::-1])[::-1]) + 1)
                            counts_with_points[(row, col)] = (counts_vectorized[row, col], (row - counts_vectorized[row, col], col))
       elif direction == 'down':
              mask |= (g == 1)
              result = np.argwhere((g > 0) & (g != 2))
              for row, col in result:
                     if row < g.shape[0] - 1:
                            counts_vectorized[row, col] = min(g.shape[0] - row - 1, np.sum(mask[row+1:, col] & np.cumprod(mask[row+1:, col])) + 1)
                            counts_with_points[(row, col)] = (counts_vectorized[row, col], (row + counts_vectorized[row, col], col))
       
       return counts_with_points

orientation = {
       0: counts_in_direction(g, 'up'),
       90: counts_in_direction(g, 'right'),
       180: counts_in_direction(g, 'down'),
       270: counts_in_direction(g, 'left')
}

display(grid_mask, orientation)


array([[8, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

{'.': 0, '|': 1, '-': 2, '/': 4, '\\': 8}

{0: {(1, 0): (1, (0, 0)),
  (1, 6): (1, (0, 6)),
  (1, 19): (1, (0, 19)),
  (1, 23): (1, (0, 23)),
  (1, 37): (1, (0, 37)),
  (1, 45): (1, (0, 45)),
  (1, 58): (1, (0, 58)),
  (1, 77): (1, (0, 77)),
  (1, 87): (1, (0, 87)),
  (1, 94): (1, (0, 94)),
  (1, 95): (1, (0, 95)),
  (1, 100): (1, (0, 100)),
  (2, 15): (2, (0, 15)),
  (2, 37): (1, (1, 37)),
  (2, 40): (2, (0, 40)),
  (2, 46): (2, (0, 46)),
  (2, 48): (2, (0, 48)),
  (2, 53): (2, (0, 53)),
  (2, 65): (2, (0, 65)),
  (2, 90): (2, (0, 90)),
  (2, 104): (2, (0, 104)),
  (2, 106): (2, (0, 106)),
  (3, 49): (3, (0, 49)),
  (3, 54): (3, (0, 54)),
  (3, 62): (3, (0, 62)),
  (3, 70): (3, (0, 70)),
  (3, 77): (2, (1, 77)),
  (3, 81): (3, (0, 81)),
  (3, 85): (3, (0, 85)),
  (3, 99): (3, (0, 99)),
  (4, 0): (4, (0, 0)),
  (4, 54): (1, (3, 54)),
  (4, 62): (4, (0, 62)),
  (4, 76): (4, (0, 76)),
  (4, 109): (4, (0, 109)),
  (5, 0): (5, (0, 0)),
  (5, 1): (5, (0, 1)),
  (5, 17): (1, (4, 17)),
  (5, 42): (5, (0, 42)),
  (5, 49): (2, (3, 49)),

In [33]:
beams = [((0, 0), 90)]

reflections = set()

activated = np.zeros_like(g)

while len(beams) > 0:
    point, direction = beams.pop()

    activated[point] = 1

    if (point, direction) in reflections:
        continue
    reflections.add((point, direction))

    current_orientation = orientation[direction]

    if point not in current_orientation:
        continue

    path_length, next_point = current_orientation[point]

    if direction == 0: 
        activated[next_point[0]:point[0]+1, point[1]] = 1
    elif direction == 90:
        activated[point[0], point[1]:next_point[1]+1] = 1
    elif direction == 180:
        activated[point[0]:next_point[0]+1, point[1]] = 1
    elif direction == 270:
        activated[point[0], next_point[1]:point[1]+1] = 1

    if next_point[0] < 0 or next_point[0] >= g.shape[0] or next_point[1] < 0 or next_point[1] >= g.shape[1]:
        # print(f"  Next point {next_point} is out of bounds")
        continue

    if g[next_point] == grid_mask['.']:
        # print(f"  Next point {next_point} is empty (boundary)")
        continue

    if g[next_point] == grid_mask['|']:
        if direction == 90 or direction == 270:
            # print(f'  Adding {next_point} to beams with direction 0 and 180')
            beams.append((next_point, 0))
            beams.append((next_point, 180))
        continue

    if g[next_point] == grid_mask['-']:
        if direction == 0 or direction == 180:
            # print(f'  Adding {next_point} to beams with direction 90 and 270')
            beams.append((next_point, 90))
            beams.append((next_point, 270))
        continue

    if g[next_point] == grid_mask['/']:
        next_direction = None
        if direction == 90: next_direction = 0
        elif direction == 180: next_direction = 270
        elif direction == 270: next_direction = 180
        elif direction == 0: next_direction = 90
        # print(f"  Adding {next_point} to beams with direction {next_direction}")
        beams.append((next_point, next_direction))

    if g[next_point] == grid_mask['\\']:
        next_direction = None
        if direction == 90: next_direction = 180
        elif direction == 180: next_direction = 90
        elif direction == 270: next_direction = 0
        elif direction == 0: next_direction = 270
        # print(f"  Adding {next_point} to beams with direction {next_direction}")
        beams.append((next_point, next_direction))

print(activated.sum())

Total length: 12715
7434
